<a href="https://colab.research.google.com/github/Dikshuy/hail_UMICaana/blob/master/Aanal_final_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pylab import *
import numpy as np
from matplotlib import pyplot as plt
import cv2
from cv_bridge import CvBridge
import rospy
from sensor_msgs.msg import Image
from keras.models import load_model

print("2")
def detect(data):
    print("4")
    bridge = CvBridge()
    image = bridge.imgmsg_to_cv2(data, "bgr8")
    print("3")

    model = load_model('model.h5')

    alphabets = ['P', 'S', 'L', 'U', 'B', 'O', 'G', 'N', 'C', 'Y', 'K', 'C', 'M', 'F', 'I', 'H', 'A', 'T']

    img = cv2.imread(image)

    img_resize = cv2.reshape(img, (64, 64))
    
    img_thresh = cv2.threshold(img_resize, 50, 255, cv2.THRESH_BINARY_INV)
    
    pixels = np.array(img_resize).reshape((64*64, 1))
    white = 0
    black  = 0 

    for i in pixels:
      if i==0:
        black +=1
      else:
        white += 1

  if black> white:
    _, img_thresh = cv2.threshold(img, 50, 255, cv2.THRESH_BINARY_INV) 
  else:
    _, img_thresh = cv2.threshold(img, 50, 255, cv2.THRESH_BINARY) 


    
    alpha = 1.4
    beta = 50

    new_img = np.zeros(img.shape, img.dtype)
    for y in range(img.shape[0]):
        for x in range(img.shape[1]):
            new_img[y, x] = np.clip(alpha * img[y, x] + beta, 0, 255)

    new_img = cv2.cvtColor(new_img, cv2.COLOR_BGR2GRAY)
    new_img = cv2.resize(new_img, (64, 64))

    contours, hierarchy = cv2.findContours(img_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    areas = np.array([int(cv2.contourArea(c)) for c in countours])
    max_index = np.argmax(areas)
    cnt=contours[max_index]
    x,y,w,h = cv2.boundingRect(cnt)

    cropped = new_img[y:y + h, x:x + w]
    cropped = cv2.resize(cropped, (128, 64))
    
    imgs = np.array(cropped).reshape(64, 128)
    img1 = imgs[:, :64]
    img2 = imgs[:, 64:]

    img1 = img1.reshape(1, 64, 64, 1)
    img2 = img2.reshape(1, 64, 64, 1)

    
    pred1 = model.predict(img1)
    pred2 = model.predict(img2)

    l1 = print(alphabets[np.argmax(pred1[0])])
    l2 = print(alphabets[np.argmax(pred2[0])])

    letters = l1 + l2
    country1 = "PUKISTAN"   #1
    country2 = "NUNNIGA"    #2
    country3 = "BULLICHYA"  #3

    if country1.find(letters) != -1:
        return 1
    elif country2.find(letters) != -1:
        return 2
    elif country3.find(letters) != -1:
        return 3
    else:
        print("SOMETHING IS WRONG")

    cv2.imshow("Image",image)
    print("aagaya")
    k = cv2.waitKey(5) & 0xFF

if __name__ == '__main__':
    
	rospy.init_node('image_gazebo', anonymous=True)
	rospy.Subscriber("/mybot/mybot/camera1/image_raw", Image, detect)
    
	rospy.spin()
